In [1]:
import torch
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
import re
import os
import threading
from PIL import Image 
import math
# import Image
%cd yolov5

C:\python_ML\Eyes-on\yolov5


In [2]:
# ------------------------- yolov5s 모델 import -------------------------
from pathlib import Path

from models.yolo import Model, attempt_load
from utils.general import check_requirements, set_logging
from utils.google_utils import attempt_download
from utils.torch_utils import select_device

# set_logging(verbose=verbose)
device = None

name1 = 'yolov5_custom_pink.pt'
fname1 = Path(name1).with_suffix('.pt')  # checkpoint filename
model1 = attempt_load(fname1, map_location=torch.device('cpu'))
model1 = model1.autoshape()  # for file/URI/PIL/cv2/np inputs and NMS
model1.conf = 0.5

name2 = 'yolov5_custom.pt'
fname2 = Path(name2).with_suffix('.pt')  # checkpoint filename
model2 = attempt_load(fname2, map_location=torch.device('cpu'))
model2 = model2.autoshape()  # for file/URI/PIL/cv2/np inputs and NMS
model2.conf = 0.5

device = select_device('0' if torch.cuda.is_available() else 'cpu') if device is None else torch.device(device)
# ------------------------- 모델 import fin -------------------------

In [4]:
# ------------------------- yolov5s 모델로 input 이미지 디텍팅 함수 -------------------------
def objectDetection(img):
    results_img = model2(img)
    results_img.crop()
#    results_img.save()

# 1. Tesseract 기존 코드

In [4]:
# ------------------------- OCR_v1 -------------------------
# OCR 버전마다 전처리 과정이 다름
def OCR_pn(img):
    img = cv2.imread(img) # 이미지 로드
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # 배경에 그레이 적용
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC) # 사이즈 정규화(키우기)
    blur = cv2.GaussianBlur(gray, (5,5), 0) # 이미지에 블러 처리

    # ------------------- 기울기 조정 start -------------------
    canny = cv2.Canny(blur, 700, 350, apertureSize = 5, L2gradient = True) # 이미지 외곽선만 추출
    lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150) # 직선 찾기

    angle = 0
    maxdim = 0
    # 각도 조정
    if not (lines is None):
        for i in lines:
            xdim = i[0][2] - i[0][0]
            ydim = i[0][3] - i[0][1]
            iangle = math.atan2(ydim, xdim)*180/np.pi
            dim = math.sqrt((xdim * xdim) + (ydim * ydim))
            if abs(angle) < 40 and maxdim < dim:
                maxdim = dim
                angle =iangle

    roih, roiw, roic = img.shape
    matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), angle, 1)
    roi = cv2.warpAffine(img, matrix, (roiw, roih))
    # ------------------- 기울기 조정 fin -------------------

    roi = cv2.resize(roi, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur_2 = cv2.GaussianBlur(roi, (5,5), 0)
    
    plate_num = ""
    try:
        # tesseract OCR 적용
        text = pytesseract.image_to_string(blur_2, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1') # whitelist: 숫자만 인식
        plate_num = re.sub('[\W_]+', '', text) # 특수문자 제거
    except:
        text = None
        
    plate_num = re.sub('[\W_]+', '', text)   
    return plate_num[-4:]


# ------------------------- OCR_v2 -------------------------
def OCR_pn2(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    plate_num = ""
    try:
        text = pytesseract.image_to_string(blur, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')
        plate_num = re.sub('[\W_]+', '', text)            
    except:
        text = None

    plate_num = re.sub('[\W_]+', '', text)   
    return plate_num[-4:]


# ------------------------- OCR_v3 -------------------------
def OCR_pn3(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # ------------------- 기울기 조정 start -------------------
    canny = cv2.Canny(blur, 700, 350, apertureSize = 5, L2gradient = True)
    lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150)

    angle = 0
    maxdim = 0
    if not (lines is None):
        for i in lines:
            xdim = i[0][2] - i[0][0]
            ydim = i[0][3] - i[0][1]
            iangle = math.atan2(ydim, xdim)*180/np.pi
            dim = math.sqrt((xdim * xdim) + (ydim * ydim))
            if abs(angle) < 40 and maxdim < dim:
                maxdim = dim
                angle =iangle

    roih, roiw, roic = img.shape
    matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), angle, 1)
    roi = cv2.warpAffine(img, matrix, (roiw, roih))
    # ------------------- 기울기 조정 fin -------------------

    gray_2 = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    gray_2 = cv2.resize(gray_2, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur_2 = cv2.GaussianBlur(gray_2, (5,5), 0)

    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(blur_2, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    blur_3 = cv2.GaussianBlur(dilation, (5,5), 0)
    
    plate_num = ""
    try:
        text = pytesseract.image_to_string(blur_3, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')
        plate_num = re.sub('[\W_]+', '', text)            
    except:
        text = None
    
    plate_num = re.sub('[\W_]+', '', text)   
    return plate_num[-4:]
            
        
# ------------------------- OCR_v4 -------------------------
def OCR_bn(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)
    blur_2 = cv2.GaussianBlur(dilation, (5,5), 0)
    
    plate_num = ""
    try:
        text = pytesseract.image_to_string(blur_2, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')            
    except:
        text = None
        
    plate_num = re.sub('[\W_]+', '', text)   
    return plate_num[-4:]

In [5]:
def OCR(img):
    img = cv2.imread(img)
    plate_num = ""
    try:
        text = pytesseract.image_to_string(img, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')            
    except:
        text = None
        
    plate_num = re.sub('[\W_]+', '', text)   
    return plate_num[-4:]

In [17]:
file_name = '69.jpg'

objectDetection(f'../img/test2/{file_name}') # yolov5s 모델로 input image 디텍팅 후 크롭된 이미지 저장

bus_num_fr_path = 'runs/hub/exp/crops/bus_num_fr/'

print(OCR(os.path.join(bus_num_fr_path, file_name)))

Saved results to runs\hub\exp

2435


In [21]:
print(f'pn:{OCR_pn(os.path.join(bus_num_fr_path, file_name))}')
print(f'pn2:{OCR_pn2(os.path.join(bus_num_fr_path, file_name))}')
print(f'pn3:{OCR_pn3(os.path.join(bus_num_fr_path, file_name))}')
print(f'bn:{OCR_bn(os.path.join(bus_num_fr_path, file_name))}')

pn:215
pn2:245
pn3:
bn:


# 2. 카카오 비전 API

In [6]:
import json

import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})


def main():
    image_path = os.path.join(bus_num_fr_path, file_name)
    appkey = 

    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr(image_path, appkey).json()
    print("[OCR] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2)))


if __name__ == "__main__":
    main()

NameError: name 'bus_num_fr_path' is not defined

# 3. Free OCR API

In [34]:
import io


def Free_OCR_API(img):

    url_api = "https://api.ocr.space/parse/image"

    file_path = img
    img = cv2.imread(file_path) 
    _, compressedimage = cv2.imencode(".jpg", img)  # (Boolean, ndarray)
                                                    # memory buffer에 이미지 넣기
    file_bytes = io.BytesIO(compressedimage)        # memory buffer에 있는 byte 형태 데이터 

    result = requests.post(url_api,
                           files={file_path: file_bytes},
                           data={"apikey": , # *수정 필요*
                                 "language": "eng"})
    result = result.content.decode() 
    result = json.loads(result)
    return result['ParsedResults'][0]['ParsedText']

In [35]:
result = Free_OCR_API('runs/hub/exp/crops/bus_num_fr/IMG_1347.jpg')
print(result)

2112



# 프레임 추출

In [ ]:
import cv2
vidcap = cv2.VideoCapture('../img/test2.mp4') # 영상 위치
success,image = vidcap.read()
count = 0
while success:
    cv2.imwrite("../img/test2/%d.jpg" % count, image)     # 이미지 저장항 위치      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

# OCR TEST code

In [10]:
base_dir = '../bus_num'
folder_list = os.listdir(base_dir)

for folder in folder_list:
    folder_dir = os.path.join(base_dir, folder)
    file_list = os.listdir(folder_dir)
    print(f'####{folder}####')
    for file in file_list:
        objectDetection(os.path.join(folder_dir,file))
        bus_num_fr_path = 'runs/hub/exp/crops/bus_num_fr/'
        try:
            result = OCR(os.path.join(bus_num_fr_path, file))
            print(result)
        except:
            print('파일이 없습니다')        

####bus_num_2015####
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp


Saved results to runs\hub\exp

2
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to r

Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp

4
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp

215
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp

2145
Saved results to runs\hub\exp


Saved results to

Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다


Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp

2
Saved results to runs\hub\exp


Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp


Saved results to runs\hub\exp


Saved results to runs\hub\exp

24
Saved results to runs\hub\exp

244
Saved results to runs\hub\exp

244
Saved results to runs\hub\exp

24
Saved results to runs\hub\exp

244
Saved resu

Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

19
Saved results to runs\hub\exp

0
Saved results to runs\hub\exp

0
Saved results to runs\hub\exp

69
Saved results to runs\hub\exp

269
Saved results to runs\hub\exp


Saved results to runs\hub\exp

2621
Saved results to runs\hub\exp

262
Saved re

Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

파일이 없습니다
Saved results to runs\hub\exp

273
Saved results to runs\hub\exp

273
Saved resu